In [28]:
import torch as t
import torchvision
from torchvision import transforms,datasets
import torch.utils.data as Data
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F

In [2]:
t.cuda.set_device(0)

In [3]:
train_transform = transforms.Compose([
    transforms.RandomCrop(32,padding=1),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914,0.4882,0.4465),(0.2023,0.1994,0.2010)),
])
test_transform = transforms.Compose([
    transforms.RandomCrop(32,padding=1),
    transforms.ToTensor(),
    transforms.Normalize((0.4914,0.4882,0.4465),(0.2023,0.1994,0.2010)),
])

In [15]:
train_data = datasets.CIFAR10(root='./cs231n/datasets',train=True,download=False,transform=train_transform)
train_loader = Data.DataLoader(train_data,batch_size=128, shuffle=True)

test_data = datasets.CIFAR10(root='./cs231n/datasets',train=False,download=False,transform=test_transform)
test_loader = Data.DataLoader(test_data,batch_size=128,shuffle=False)


In [55]:
#因为用的是Cifar10数据集所以将网络的规模设置的比较小
class alexnet_bianzhong(nn.Module):
    def __init__(self):
        super(alexnet_bianzhong,self).__init__()
        self.feature = nn.Sequential(
            nn.Conv2d(3,96,5,padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            
            nn.Conv2d(96,256,5,padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            
            nn.Conv2d(256,384,3,padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384,384,3,padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384,256,3,padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
        )
        self.classifier = nn.Sequential(
            nn.Linear(256*4*4,200),
            nn.Dropout(0.5),
            nn.Linear(200,200),
            nn.Dropout(0.5),
            nn.Linear(200,10),
        )
        
    def forward(self,x):
        x = self.feature(x)
        x = x.view(x.size()[0],-1)
        x = self.classifier(x)
        return x

In [11]:
print(256*16)

4096


In [56]:
def train_process(model,optimizer,epoches):
    model.train()
    for e in range(epoches):
        sum_loss = 0.0
        for i, t in enumerate(train_loader):
            x, y = t
            x, y = Variable(x).cuda(),Variable(y).cuda()
#             print(x.shape)
            
            optimizer.zero_grad()
            
            scores = model(x)
            loss = F.cross_entropy(scores,y)
            
            loss.backward()
            optimizer.step()
            sum_loss += loss.item()
            if i % 20 == 0:
                print('Iteration:%d batch:%d %.03f '%(e,i,sum_loss))
                sum_loss = 0.0
    return 

In [65]:
def test_process(model):
    model.eval()
    num, correct= 0, 0
    for i,(x,y) in enumerate(test_loader):
        x, y = Variable(x).cuda(), Variable(y).cuda()
        scores = model(x)
        _, pre = scores.max(1)
        num += scores.size()[0]
        correct += (pre==y).sum()
        acc = float(correct)/num
        print('%d %.03f'%(i,acc*100))

In [58]:
model =alexnet_bianzhong().cuda()
learning_rate = 1e-3
optimizer = optim.SGD(model.parameters(),lr=learning_rate,momentum=0.9,weight_decay=5e-4)
# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer)

In [59]:
train_process(model,optimizer,epoches=10)

Iteration:0 batch:0 230.655 
Iteration:0 batch:20 4605.149 
Iteration:0 batch:40 4610.941 
Iteration:0 batch:60 4608.252 
Iteration:0 batch:80 4603.929 
Iteration:0 batch:100 4605.952 
Iteration:0 batch:120 4606.651 
Iteration:0 batch:140 4603.148 
Iteration:0 batch:160 4601.970 
Iteration:0 batch:180 4601.101 
Iteration:0 batch:200 4601.234 
Iteration:0 batch:220 4599.508 
Iteration:0 batch:240 4602.357 
Iteration:0 batch:260 4596.796 
Iteration:0 batch:280 4598.846 
Iteration:0 batch:300 4593.542 
Iteration:0 batch:320 4592.732 
Iteration:0 batch:340 4588.928 
Iteration:0 batch:360 4589.317 
Iteration:0 batch:380 4585.502 
Iteration:1 batch:0 229.279 
Iteration:1 batch:20 4579.183 
Iteration:1 batch:40 4564.944 
Iteration:1 batch:60 4561.756 
Iteration:1 batch:80 4546.027 
Iteration:1 batch:100 4532.317 
Iteration:1 batch:120 4495.621 
Iteration:1 batch:140 4471.215 
Iteration:1 batch:160 4450.271 
Iteration:1 batch:180 4376.242 
Iteration:1 batch:200 4345.869 
Iteration:1 batch:220 

In [66]:
test_process(model)

0 55.469
1 51.953
2 53.125
3 53.711
4 54.219
5 53.516
6 53.237
7 54.688
8 54.514
9 54.922
10 54.759
11 55.339
12 55.228
13 55.190
14 55.365
15 55.420
16 55.469
17 55.382
18 55.304
19 55.039
20 54.836
21 54.545
22 54.620
23 54.850
24 55.094
25 55.048
26 54.832
27 54.799
28 54.688
29 54.714
30 54.738
31 54.565
32 54.593
33 54.802
34 54.665
35 54.536
36 54.540
37 54.523
38 54.567
39 54.395
40 54.325
41 54.222
42 54.161
43 54.208
44 54.358
45 54.246
46 54.205
47 54.102
48 54.050
49 54.031
50 53.983
51 53.816
52 53.729
53 53.791
54 53.594
55 53.683
56 53.618
57 53.704
58 53.774
59 53.659
60 53.676
61 53.616
62 53.646
63 53.699
64 53.558
65 53.551
66 53.556
67 53.493
68 53.499
69 53.504
70 53.499
71 53.570
72 53.671
73 53.653
74 53.781
75 53.834
76 53.764
77 53.716
78 53.700
